In [6]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import json

In [7]:
# 1. Настройки (должны совпадать с вашим скриптом валидации)
MLFLOW_URI = "http://127.0.0.1:5001"
TARGET_RUN_ID = "8f29da97ce2f4612a8b14cf7bccaf648"  # Вставьте сюда ID из вывода валидации
EXPERIMENT_ID = "3"

# Инициализация клиента
mlflow.set_tracking_uri(MLFLOW_URI)
client = MlflowClient(tracking_uri=MLFLOW_URI)

In [ ]:
def get_run_details(run_id):
    """Получает базовые метрики и параметры Run-а."""
    run = client.get_run(run_id)
    print(f"--- Run: {run_id} ---")
    print(f"Status: {run.info.status}")
    print(f"Params: {json.dumps(run.data.params, indent=2)}")
    print(f"Metrics: {json.dumps(run.data.metrics, indent=2)}")
    return run

def get_llm_traces(run_id, experiment_id):
    """Извлекает LLM трейсы (входы, выходы, промежуточные шаги)."""
    # search_traces возвращает список объектов Trace
    traces = client.search_traces(
        experiment_ids=[experiment_id], # Можно оставить пустым, если фильтруем по run_id
        filter_string=f"attributes.run_id = '{run_id}'"
    )
    extracted_data = []
    print(f"\n--- Найдено трейсов: {len(traces)} ---")
    
    for trace in traces:
        # Каждый trace содержит data (spans) и info
        # Обычно корневой span (первый) содержит общий вход и выход RAG
        root_span = trace.data.spans[0]
        trace_info = {
            "request_id": trace.info.request_id,
            "timestamp": trace.info.timestamp_ms,
            "latency_ms": trace.info.execution_time_ms,
            "status": trace.info.status,
            # Входы и выходы могут быть строками или JSON
            "input": root_span.inputs, 
            "output": root_span.outputs,
        }
        
        # Если нужно углубиться в детали (например, retrieved documents)
        # нужно перебирать spans внутри trace.data.spans
        # Пример поиска span с именем "Retriever" (название зависит от вашей реализации)
        retriever_span = next((s for s in trace.data.spans if "retriev" in s.name.lower()), None)
        if retriever_span:
            trace_info["retrieved_docs"] = retriever_span.outputs
        extracted_data.append(trace_info)
    return extracted_data

In [9]:
# 1. Получаем общую информацию
run = get_run_details(TARGET_RUN_ID)
run

--- Run: 8f29da97ce2f4612a8b14cf7bccaf648 ---
Status: FINISHED
Params: {
  "validation_timestamp": "2025-12-09T18:03:29.337680",
  "run_type": "validation"
}
Metrics: {}


<Run: data=<RunData: metrics={}, params={'run_type': 'validation', 'validation_timestamp': '2025-12-09T18:03:29.337680'}, tags={'mlflow.runName': 'validation_20251209_180328',
 'mlflow.source.git.commit': 'cf2f4469651cb71f1691cfa96b1d6d03c49c1174',
 'mlflow.source.name': 'src/validation/run.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'i.romaikin'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/3/8f29da97ce2f4612a8b14cf7bccaf648/artifacts', end_time=1765292672435, experiment_id='3', lifecycle_stage='active', run_id='8f29da97ce2f4612a8b14cf7bccaf648', run_name='validation_20251209_180328', start_time=1765292609315, status='FINISHED', user_id='i.romaikin'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[]>>

In [10]:
# 2. Получаем контент трейсов
traces_content = get_llm_traces(TARGET_RUN_ID, EXPERIMENT_ID)
traces_content


--- Найдено трейсов: 4 ---


[{'request_id': 'tr-b660fb9af941f0e1789c7d0e596cc36f',
  'timestamp': 1765292655128,
  'latency_ms': 17160,
  'status': <TraceStatus.OK: 'OK'>,
  'input': {'query': 'Как создать кастомную тень для View, чтобы она имела произвольную форму, учитывая ограничения свойства android:elevation?',
   'documents': [],
   'doc_ids': '',
   'answer': ''},
  'output': {'query': 'Как создать пользовательскую тень произвольной формы для View, учитывая ограничения свойства android:elevation?',
   'documents': [{'id': 558542,
     'author': 'mobileSimbirSoft',
     'url': 'https://habr.com/ru/post/558542/',
     'title': 'То, чего нам так не хватало: Render Effect в Android 12',
     'document_id': 5,
     'chunk_id': 6,
     'content': 'REPEAT. Изображение повторяется горизонтально и вертикально. На изображении заметно, как темная вода из нижней части изображения отразилась на верхней границе изображения.\n DECAL (появился в API 31). Отрисовка shader-а только в пределах границ. Можно заметить, что на 

In [ ]:
def traces_to_dataframe(traces: list) -> pd.DataFrame:
    """Преобразует список трейсов MLflow в DataFrame."""
    data = []
    for trace in traces:
        request_id = trace.get('request_id')
        inp = trace.get('input', {})
        out = trace.get('output', {})
        question = inp.get('query', '')
        answer = out.get('answer', '')
        
        # 3. Обработка документов
        documents = out.get('documents', [])
        
        if documents:
            # Если документов нет, добавляем запись только с вопросом и ответом
            data.append({
                'request_id': request_id,
                'question': question,
                'answer': answer,
                'document_id': [doc.get('document_id') for doc in documents],
                'chunk_id': [doc.get('chunk_id') for doc in documents],
                'content': [doc.get('content') for doc in documents]
            })
    df = pd.DataFrame(data)
    return df

# Создаем DataFrame
df = traces_to_dataframe(traces_content)
df

,request_id,question,answer,document_id,chunk_id,content
0,tr-b660fb9af941f0e1789c7d0e596cc36f,"Как создать кастомную тень для View, чтобы она имела произвольную форму, учитывая ограничения св...",**Как создать пользовательскую тень произвольной формы для View**\n\n1. **Создайте графический р...,5,6,"REPEAT. Изображение повторяется горизонтально и вертикально. На изображении заметно, как темная ..."
1,tr-b660fb9af941f0e1789c7d0e596cc36f,"Как создать кастомную тень для View, чтобы она имела произвольную форму, учитывая ограничения св...",**Как создать пользовательскую тень произвольной формы для View**\n\n1. **Создайте графический р...,6,7,"Кастомную тень можно создать и с помощью xml, прописав с градиентом:\n[code]\n \n[/code]\nМы пол..."
2,tr-71da94c6e7948a253982c5428825dae8,Какую новую форму оценки требуют абитуриенты Университета Чикаго с этой осени?,Новые требования к абитуриентам Университета Чикаго к началу осеннего семестра включают обязател...,1,1,"Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо с..."
3,tr-71da94c6e7948a253982c5428825dae8,Какую новую форму оценки требуют абитуриенты Университета Чикаго с этой осени?,Новые требования к абитуриентам Университета Чикаго к началу осеннего семестра включают обязател...,6,29,"Чтобы полностью исключить шину UPI из обработки и передачи данных, мы поступим самым простым и н..."
4,tr-b8854fdd3c08f65b6dde8f8bc508d352,"Какие ограничения у свойства android:elevation при создании теней в Android, и как можно получит...",,5,6,"REPEAT. Изображение повторяется горизонтально и вертикально. На изображении заметно, как темная ..."
5,tr-b8854fdd3c08f65b6dde8f8bc508d352,"Какие ограничения у свойства android:elevation при создании теней в Android, и как можно получит...",,6,7,"Кастомную тень можно создать и с помощью xml, прописав с градиентом:\n[code]\n \n[/code]\nМы пол..."
6,tr-b6b7066273722baa79262943f9d8a986,"Сколько презентаций PowerPoint, по оценкам Microsoft, демонстрируется в мире ежедневно?",По оценкам Microsoft ежедневно в мире демонстрируется примерно **30 млн презентаций PowerPoint**...,1,1,"Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо с..."
7,tr-b6b7066273722baa79262943f9d8a986,"Сколько презентаций PowerPoint, по оценкам Microsoft, демонстрируется в мире ежедневно?",По оценкам Microsoft ежедневно в мире демонстрируется примерно **30 млн презентаций PowerPoint**...,7,30,Windows Server 2016:\nWrite (percent)\nодин CPU — IOPS\nдва CPU — IOPS\n0 (read)\n1132231.01\n59...
